In [1]:
import requests
import pandas as pd
from datetime import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import formatdate
from email.mime.base import MIMEBase
from email.encoders import encode_base64

In [21]:
today = str(datetime.now().date()).replace("-", "")
today

'20220505'

In [22]:
def get_stock_indices(date):  ### date type : str(),,, ex) '2010722'

    url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

    data = {'bld': 'dbms/MDC/STAT/standard/MDCSTAT01501',
            'locale': 'ko_KR',
            'mktId': 'ALL',
            'trdDd': date,
            'share': '1',
            'money': '1',
            'csvxls_isNo': 'false'}

    response = requests.post(url, data=data)  ### get이 아님에 유의
    stock_data = response.json()['OutBlock_1']  ### 불러온 정보를 json으로 추출하면 dict()구조인데 원하는 정보는 key:'output'에 있다.

    return stock_data

In [30]:
def send_stock_info(today):
    smtp = smtplib.SMTP_SSL("smtp.gmail.com", 465)
    id = "Gmail Address"
    pw = "app password"  # fytqladsnkwfdorg
    smtp.login(id, pw)
    email_from = id
    email_to = "charkim@naver.com"
    email_date = formatdate(localtime=True)
    email_subject = today + " KOSPI 주식 현황"
    email_message_html = """안녕하세요. <br>
    <h1> 오늘자 KOSPI 주식 현황을 보고서를 첨부합니다. </h1>
    <p><span style="font-weight:bold">감사합니다.</span> </p>
    """
    msg = MIMEMultipart('mixed')
    # 보내는 사람
    msg['From'] = email_from
    # 받는 사람
    msg['To'] = email_to
    # 날짜
    msg['Date'] = email_date
    # 제목
    msg['Subject'] = email_subject
    # 본문 추가 (html형식으로 인코딩 방식은 utf-8)
    msg.attach(MIMEText(email_message_html, "html", _charset="utf-8"))
    # msg.attach(MIMEText(email_message))

    # 실제 파일 경로를 open함수로 read 한다.
    filename = "kospi_" + today + ".csv"
    target_file = "./" + filename
    with open(target_file, "rb") as attach_file:
        # application, octet-stream은 모든 종류의 파일을 전송할 때 사용하는 타입
        file_data = MIMEBase("application", "octet-stream")
        file_data.set_payload(attach_file.read())
        encode_base64(file_data)
        # add_header에 filename를 전달해야 첨부된 파일명이 보입니다.
        file_data.add_header('Content-Disposition', 'attachment', filename=filename)
        msg.attach(file_data)

    smtp.sendmail(id, email_to, msg.as_string())

In [31]:
stock_data = get_stock_indices(today)
data = pd.DataFrame(stock_data)

In [32]:
data

,ISU_SRT_CD,ISU_ABBRV,MKT_NM,SECT_TP_NM,TDD_CLSPRC,FLUC_TP_CD,CMPPREVDD_PRC,FLUC_RT,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,ACC_TRDVOL,ACC_TRDVAL,MKTCAP,LIST_SHRS,MKT_ID
0,060310,3S,KOSDAQ,중견기업부,-,3,-,-,-,-,-,-,-,-,"46,271,513",KSQ
1,095570,AJ네트웍스,KOSPI,,-,3,-,-,-,-,-,-,-,-,"46,822,295",STK
2,006840,AK홀딩스,KOSPI,,-,3,-,-,-,-,-,-,-,-,"13,247,561",STK
3,054620,APS홀딩스,KOSDAQ,중견기업부,-,3,-,-,-,-,-,-,-,-,"20,394,221",KSQ
4,265520,AP시스템,KOSDAQ,우량기업부,-,3,-,-,-,-,-,-,-,-,"15,281,421",KSQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2619,000547,흥국화재2우B,KOSPI,,-,3,-,-,-,-,-,-,-,-,"153,600",STK
2620,000545,흥국화재우,KOSPI,,-,3,-,-,-,-,-,-,-,-,"768,000",STK
2621,003280,흥아해운,KOSPI,,-,3,-,-,-,-,-,-,-,-,"240,424,899",STK
2622,037440,희림,KOSDAQ,우량기업부,-,3,-,-,-,-,-,-,-,-,"13,922,475",KSQ


In [33]:
data_kospi = data[data.MKT_NM == 'KOSPI'].sort_values(by=['ISU_SRT_CD'])

In [34]:
data_kospi

,ISU_SRT_CD,ISU_ABBRV,MKT_NM,SECT_TP_NM,TDD_CLSPRC,FLUC_TP_CD,CMPPREVDD_PRC,FLUC_RT,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,ACC_TRDVOL,ACC_TRDVAL,MKTCAP,LIST_SHRS,MKT_ID
631,000020,동화약품,KOSPI,,-,3,-,-,-,-,-,-,-,-,"27,931,470",STK
136,000040,KR모터스,KOSPI,,-,3,-,-,-,-,-,-,-,-,"96,137,368",STK
290,000050,경방,KOSPI,,-,3,-,-,-,-,-,-,-,-,"27,415,270",STK
789,000060,메리츠화재,KOSPI,,-,3,-,-,-,-,-,-,-,-,"120,625,000",STK
997,000070,삼양홀딩스,KOSPI,,-,3,-,-,-,-,-,-,-,-,"8,564,271",STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,402340,SK스퀘어,KOSPI,,-,3,-,-,-,-,-,-,-,-,"141,467,571",STK
1221,404990,신한서부티엔디리츠,KOSPI,,-,3,-,-,-,-,-,-,-,-,"55,955,884",STK
2079,417310,코람코더원리츠,KOSPI,,-,3,-,-,-,-,-,-,-,-,"40,400,000",STK
1528,900140,엘브이엠씨홀딩스,KOSPI,,-,3,-,-,-,-,-,-,-,-,"99,455,297",STK


In [35]:
final = "kospi_" + today + ".csv"
data_kospi.to_csv(final, index=None, header=True)

In [36]:
send_stock_info(today)